# Case Study: Borden

The Borden dataset is a simple, low-resource DEM that forms the basis of the
Borden Benchmark test case.

The test case is based on the original field experiment and hydraulic parameters of
[Abdul and Gillham](https://doi.org/10.1016/0022-1694(89)90177-7), and is used by
[Kollet et al.](https://doi.org/10.1002/2016WR019191) for integrated hydrologic
model intercomparison.

In [1]:
import tinerator as tin
from tinerator import examples

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


## Borden DEMs

The DEM is available in both 1 meter and 0.5 meter (50 cm) resolutions:

In [4]:
dem_1m = tin.gis.load_raster(examples.borden.dem_100cm)
dem_1m.fill_depressions()

dem_50cm = tin.gis.load_raster(examples.borden.dem_50cm)
dem_50cm.fill_depressions()

[16:26:53] Could not parse CRS "". Defaulting to "EPSG:32601"                  ]8;id=1631831213.321457-649426;file:///Users/livingston/dev/lanl/tinerator/tinerator/logging.py\logging.py]8;;\:126


A Priority-Flood (Zhou2016 version)
C Zhou, G., Sun, Z., Fu, S., 2016. An efficient variant of the Priority-Flood algorithm for filling depressions in raster digital elevation models. Computers & Geosciences 90, Part A, 87 – 96. doi:http://dx.doi.org/10.1016/j.cageo.2016.02.021

t Zhou2016 wall-time = 0.000188723 s

A Barnes (2014) Flat Resolution Flat Mask Generation
C Barnes, R., Lehman, C., Mulla, D., 2014a. An efficient assignment of drainage direction over flat surfaces in raster digital elevation models. Computers & Geosciences 62, 128–135. doi:10.1016/j.cageo.2013.01.009

t Succeeded in = 1.0984e-05 s
p Setting up labels matrix...
p Setting up flat resolution mask...
p Searching for flats...
t Succeeded in = 0.00121137 s
m Cells with no flow direction = 61
m Low edge cells               = 4
m High edge cells              = 61
p Labeling flats...
m Unique flats = 5
p Removing flats without outlets from the queue...
  The flat height vector will require approximately 0MB of RAM.


           Could not parse CRS "". Defaulting to "EPSG:32601"                  ]8;id=1631831213.355318-267965;file:///Users/livingston/dev/lanl/tinerator/tinerator/logging.py\logging.py]8;;\:126

[==========================================        ] (85% - 0.0s - 1 threads)W Cells inappropriately raised above surrounding terrain = 0
t Succeeded in = 0.00127794 s

A Priority-Flood (Zhou2016 version)
C Zhou, G., Sun, Z., Fu, S., 2016. An efficient variant of the Priority-Flood algorithm for filling depressions in raster digital elevation models. Computers & Geosciences 90, Part A, 87 – 96. doi:http://dx.doi.org/10.1016/j.cageo.2016.02.021

t Zhou2016 wall-time = 0.000434173 s

A Barnes (2014) Flat Resolution Flat Mask Generation
C Barnes, R., Lehman, C., Mulla, D., 2014a. An efficient assignment of drainage direction over flat surfaces in raster digital elevation models. Computers & Geosciences 62, 128–135. doi:10.1016/j.cageo.2013.01.009

t Succeeded in = 3.9829e-05 s
p Setting up labels matrix...
p Setting up flat resolution mask...
p Searching for flats...
t Succeeded in = 0.00125527 s================ ] (99% - 0.0s - 1 threads)
m Cells with no flow direction = 149
m Low edge ce

In [3]:
dem_1m.plot()

In [5]:
dem_50cm.plot()

## Generating the surface mesh

In [6]:
ws_flow = tin.gis.watershed_delineation(dem_50cm, threshold=100., method='D8')
ws_flow.plot(layers=[dem_50cm])


A O'Callaghan (1984)/Marks (1984) D8/D4 Flow Accumulation
C O'Callaghan, J.F., Mark, D.M., 1984. The Extraction of Drainage Networks from Digital Elevation Data. Computer vision, graphics, and image processing 28, 323--344.

c topology = D8
[================================================= ] (99% - 0.0s - 1 threads)
A Generic Flow Accumulation Algorithm
p Creating dependencies array...

d Source cells found = 857
p Calculating flow accumulation...
t Wall-time       = 0.00133079 s


In [7]:
triangular_surface = tin.meshing.triangulate(
    dem_50cm,
    min_edge_length=0.01,
    max_edge_length=0.1,
    method='jigsaw',
    refinement_feature=ws_flow,
    scaling_type='relative',
)

triangular_surface.plot()

# Extruding the mesh

The layering schema is:
  * Layer 1: 50 cm thick, with 5 sublayers
  * Layer 2: 1 meter thick, with 5 sublayers in 20 cm increments
  * Layer 3: flat bottom at `z = 0 m`, with 5 sublayers

In [8]:
layers = [
    ("constant", 0.5, 5, 1),
    ("constant", 1., 5, 2),
    ("snapped", 0., 5, 3),
]

volume_mesh = tin.meshing.extrude_mesh(triangular_surface, layers)
volume_mesh.plot()

# Generating sets

To generate sets, we first build a surface mesh with `vol_mesh.surface_mesh()`. This provides us with the necessarily helper functions to get started constructing side and point sets.

Some default sets are available, like `top_faces`, `top_nodes`, `bottom_faces`, `bottom_nodes`, `side_faces`, and `side_nodes`.

We can pass these sets into the `vol_mesh.view()` method to see them.

In [10]:
surface_mesh = volume_mesh.surface_mesh()

# Extract top and bottom faces, and side nodes
top_faces = surface_mesh.top_faces
bottom_faces = surface_mesh.bottom_faces
side_faces = surface_mesh.side_faces

print('Sets')
print('----')
print(top_faces)
print(bottom_faces)
print(side_faces)

# View them
volume_mesh.plot(sets=(top_faces, bottom_faces, side_faces))

Sets
----
SideSet<name=TopFaces, num_faces=2413>
SideSet<name=BottomFaces, num_faces=2413>
SideSet<name=SideFaces, num_faces=1335>


# Advanced set operations

More advanced set functionality is provided. With the `discretize_sides` method, we can pass in a set of nodes (in clockwise ordering) to capture the faces along that perimeter. The nodes are (x, y) pairs, assumed to be in the same CRS as the mesh.

Using the `at_layer` argument allows you to specify a specific layer instead of every side.

There are also set functions like `remove`, `join`, and `intersection`, which allow you to interact with sets for more complex generation.

In [11]:
# Generates two sets:
#  1 - Outlet (between the line segment in `outlet_pts`)
#  2 - No flow (everywhere else)

outlet_pts = [
    (71.9, 4.7),
    (67.3, 2.1),
]

# Returns list of two sets
outlet = surface_mesh.discretize_sides(
    outlet_pts, close_ends=True, #at_layer=(1, 1), set_name_prefix="Outlet"
)

# Change the set names
outlet[0].name = "Outlet"
outlet[1].name = "NoFlow"

print('Outlet sets: ')
print('-------------')
print(outlet)

# Visualize
volume_mesh.plot(sets=[outlet])

[16:28:29] Generating side set for sublayer 1 of 15                       ]8;id=1631831309.28668-10389;file:///Users/livingston/dev/lanl/tinerator/tinerator/meshing/surface_mesh.py\surface_mesh.py]8;;\:374

           Generating side set for sublayer 2 of 15                       ]8;id=1631831309.310144-994933;file:///Users/livingston/dev/lanl/tinerator/tinerator/meshing/surface_mesh.py\surface_mesh.py]8;;\:374

           Generating side set for sublayer 3 of 15                       ]8;id=1631831309.332048-76506;file:///Users/livingston/dev/lanl/tinerator/tinerator/meshing/surface_mesh.py\surface_mesh.py]8;;\:374

           Generating side set for sublayer 4 of 15                       ]8;id=1631831309.3637671-490314;file:///Users/livingston/dev/lanl/tinerator/tinerator/meshing/surface_mesh.py\surface_mesh.py]8;;\:374

           Generating side set for sublayer 5 of 15                       ]8;id=1631831309.3948789-70296;file:///Users/livingston/dev/lanl/tinerator/tinerator/meshing/surface_mesh.py\surface_mesh.py]8;;\:374

...

           Generating side set for sublayer 12 of 15                      ]8;id=1631831309.650934-464134;file:///Users/livingston/dev/lanl/tinerator/tinerator/meshing/surface_mesh.py\surface_mesh.py]8;;\:374

           Generating side set for sublayer 13 of 15                      ]8;id=1631831309.6959722-693327;file:///Users/livingston/dev/lanl/tinerator/tinerator/meshing/surface_mesh.py\surface_mesh.py]8;;\:374

           Generating side set for sublayer 14 of 15                      ]8;id=1631831309.7416708-165190;file:///Users/livingston/dev/lanl/tinerator/tinerator/meshing/surface_mesh.py\surface_mesh.py]8;;\:374

           Generating side set for sublayer 15 of 15                      ]8;id=1631831309.791775-169201;file:///Users/livingston/dev/lanl/tinerator/tinerator/meshing/surface_mesh.py\surface_mesh.py]8;;\:374

Outlet sets: 
-------------
[SideSet<name=Outlet, num_faces=1230>, SideSet<name=NoFlow, num_faces=105>]


# Writing to disk

Sets can be written to disk in the ExodusII meshing format:

In [15]:
volume_mesh.save("Borden.exo", sets=[outlet, top_faces, bottom_faces])

Outlet: min = 700; max = 33997
NoFlow: min = 11; max = 34268
TopFaces: min = 1; max = 2286
BottomFaces: min = 32005; max = 34290

You are using exodus.py v 1.20.2 (seacas-py3), a python wrapper of some of the exodus library.

Copyright (c) 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021 National Technology &
Engineering Solutions of Sandia, LLC (NTESS).  Under the terms of
Contract DE-NA0003525 with NTESS, the U.S. Government retains certain
rights in this software.

Opening exodus file: Borden.exo
Closing exodus file: Borden.exo
EXODUS write was successful.
